# Import the exports from both logistic regression and ML classifications
# Perform comparisons on the two methods

In [1]:
%reset -f
%matplotlib inline

import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
dirToInclude=parentdir +'/features/'
sys.path.insert(0,dirToInclude)


import pandas
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

import scipy as sp
import theano.tensor as t

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
rf_results = pd.read_csv("Outputs/rf_results.csv")
reg_results = pd.read_csv("Outputs/mlogreg_results.csv")

In [22]:
rf_results.head(3)

,Unnamed: 0,patientid,diag,rfpred,CMalaria,Enceph,Mening,Other,headache,age,temp,date,clinjaund,clintonsil,clinhepato,clinaids,clinhemmor,clinorcand,clinearinf,clinresp,clinlymph,clindehyd,clinoedem,clinablung,clincyan,clincapref,glasgtot,dbp,sbp,resp,card,weight,height,muac,mhhosp,mhchronstat,mhdevsp,conv,abdpain,vom,diar,muscle,monthday,incirrit,inchead,incphoto,incfont,incneuro,incseiza,incseizh,incpurp
0,0,244.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,53.0,37.2,661.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,50.0,86.0,38.0,149.0,13.5,102.0,13.8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,38.0,260.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,45.0,94.0,31.0,144.0,10.0,86.5,14.4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,162.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2,66.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,104.0,38.3,179.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,51.0,101.0,50.0,107.0,23.0,125.0,18.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [23]:
reg_results.head(3)

,Unnamed: 0,p_bact,pred_bact,diag,CMalaria,Enceph,Mening,glasgtot,sbp,patientid,p_vir,pred_vir,p_mal&bact,pred_mal&bact,p_tbcrypto,pred_tbcrypto,p_other,pred_other,mlogregpred,p_max
0,0,0.039135,0,0.0,1.0,0.0,0.0,9.0,86.0,244.0,0.008062,0,0.034391,0,0.000001,0,0.246984,0,0,0.0
1,1,0.040313,0,0.0,1.0,0.0,0.0,7.0,94.0,100.0,0.008630,0,0.039055,0,0.000006,0,0.242427,0,0,0.0
2,2,0.039994,0,5.0,1.0,0.0,0.0,6.0,101.0,66.0,0.007997,0,0.040509,0,0.000014,0,0.239728,0,0,0.0


In [13]:
analysis = pd.DataFrame(data = {'diag': reg_results['diag'] ,'rf_pred': rf_results['rfpred'] ,'reg_pred': reg_results['mlogregpred']})

In [24]:
analysis

,diag,rf_pred,reg_pred
0,0.0,0.0,0
1,0.0,0.0,0
2,5.0,0.0,0
3,0.0,0.0,0
4,0.0,0.0,0
5,0.0,0.0,0
6,0.0,5.0,5
7,1.0,1.0,5
8,0.0,0.0,0
9,1.0,5.0,5


In [16]:
sum(analysis['diag'] == analysis['rf_pred'])

220

In [17]:
sum(analysis['diag'] == analysis['reg_pred'])

225